In [ ]:
# Setup chunk to install and load required packages
knitr::opts_chunk$set(warning = FALSE, message = FALSE, include = FALSE)
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'glmnet',
               'randomForest', 'xgboost','patchwork',
               'paletteer', 'here', 'doParallel', 'summarytools')

## Regression

*Supervised* machine learning techniques involve training a model to operate on a set of *features* and predict a *label* using a dataset that includes some already-known label values.
The training process *fits* the features to the known labels to define a general function that can be applied to new features for which the labels are unknown, and predict them.
You can think of this function like this, in which ***y*** represents the label we want to predict and ***x*** represents the features the model uses to predict it.

$$y = f(x)$$

In most cases, *x* is actually a *vector* that consists of multiple feature values, so to be a little more precise, the function could be expressed like this:

$$y = f([x_1, x_2, x_3, ...])$$

The goal of training the model is to find a function that performs some kind of calculation to the *x* values that produces the result *y*.
We do this by applying a machine learning *algorithm* that tries to fit the *x* values to a calculation that produces *y* reasonably accurately for all of the cases in the training dataset.

There are lots of machine learning algorithms for supervised learning, and they can be broadly divided into two types:

-   ***Regression*** **algorithms**: Algorithms that predict a *y* value that is a numeric value, such as the price of a house or the number of sales transactions.
-   ***Classification*** **algorithms**: Algorithms that predict to which category, or *class*, an observation belongs. The *y* value in a classification model is a vector of probability values between 0 and 1, one for each class, indicating the probability of the observation belonging to each class.

In this notebook, we'll focus on *regression*, using an example based on a real study in which data for a bicycle sharing scheme was collected and used to predict the number of rentals based on seasonality and weather conditions.
We'll use a simplified version of the dataset from that study.

> **Citation**: The data used in this exercise is derived from [Capital Bikeshare](https://www.capitalbikeshare.com/system-data) and is used in accordance with the published [license agreement](https://www.capitalbikeshare.com/data-license-agreement).

### Explore the Data

The first step in any machine learning project is to `explore the data` that you will use to train a model.
The goal of this exploration is to try to understand the `relationships` between its attributes; in particular, any apparent correlation between the *features* and the *label* your model will try to predict.

This may require some work to detect and `fix issues in the data` (such as dealing with missing values, errors, or outlier values), `deriving new feature columns` by transforming or combining existing features (a process known as *feature engineering*), `normalizing` numeric features (values you can measure or count) so they're on a similar scale, and `encoding categorical features` (values that represent discrete categories) as numeric indicators.

Let's start by loading the bicycle sharing data as a `tibble` and viewing the first few rows.

In [1]:
# Load the core tidyverse and make it available in your current R session.
library(tidyverse)

# Import the data into the R session
bike_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv", show_col_types = FALSE)

# View first few rows
bike_data %>% 
  slice_head(n = 7)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,rentals
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.1604460,331
2,1/2/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.2485390,131
3,1/3/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.2483090,120
4,1/4/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.1602960,108
5,1/5/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.1869000,82
6,1/6/2011,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,88
7,1/7/2011,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.1687260,148


Sometimes, we may want some little more information on our data.
We can have a look at the `data` and `its structure` by using the [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) function.

In [ ]:
# Take a quick glance at the data
glimpse(bike_data)

Good job!�
���

We can observe that `glimpse()` will give you the total number of rows (observations) and columns (variables), then, the first few entries of each variable in a row after the variable name.
In addition, the *data type* of the variable is given immediately after each variable's name inside `< >`.

The data consists of *731 rows* the following *14 columns*:

-   **instant**: A unique row identifier

-   **dteday**: The date on which the data was observed - in this case, the data was collected daily; so there's one row per date.

-   **season**: A numerically encoded value indicating the season (1-spring, 2-summer, 3-fall, 4-winter)

-   **yr**: The year of the study in which the observation was made (the study took place over two years (year 0 represents 2011, and year 1 represents 2012)

-   **mnth**: The calendar month in which the observation was made (1-January ... 12-December)

-   **holiday**: A binary value indicating whether or not the observation was made on a public holiday)

-   **weekday**: The day of the week on which the observation was made (0-Sunday ... 6-Saturday)

-   **workingday**: A binary value indicating whether or not the day is a working day (not a weekend or holiday)

-   **weathersit**: A categorical value indicating the weather situation (1-clear, 2-mist/cloud, 3-light rain/snow, 4-heavy rain/hail/snow/fog)

-   **temp**: The temperature in celsius (normalized)

-   **atemp**: The apparent ("feels-like") temperature in celsius (normalized)

-   **hum**: The humidity level (normalized)

-   **windspeed**: The windspeed (normalized)

-   **rentals**: The number of bicycle rentals recorded.

In this dataset, `rentals` represents the `label` (the $y$ value) our model must be trained to predict.
The other columns are potential features ($x$ values).

As mentioned previously, you can perform some *feature engineering* to combine or derive new features.
For example, let's add a new column named **day** to the data frame by extracting the day component from the existing **dteday** column.
The new column represents the day of the month from 1 to 31.

From the output of *glimpse(),* you'll realize that the **dteday** column is stored as a `character` vector.
So, we'll first need to transform this to a date object.

> [Lubridate](https://lubridate.tidyverse.org/), a package in the tidyverse, provides tools that make it easier to parse and manipulate dates.

In [ ]:
# load lubridate into the R session
library(lubridate)

# Parse dates then extract days
bike_data <- bike_data %>%
  # Parse dates
  mutate(dteday = mdy(dteday)) %>% 
  #Get day
  mutate(day = day(dteday))

# extract the first 10 rows
bike_data %>% 
  slice_head(n = 10)

OK, let's start our analysis of the data by examining a few key descriptive statistics.
We can use the `summarytools::descr()` function to neatly and quickly summarize the numeric features as well as the **rentals** label column.

In [ ]:
# load package into the R session
library(summarytools)

# Obtain summary stats for feature and label columns
bike_data %>% 
  # Select features and label
  select(c(temp, atemp, hum, windspeed, rentals)) %>% 
  # Summary stats
  descr(order = "preserve",
        stats = c('mean', 'sd', 'min', 'q1', 'med', 'q3', 'max'),
        round.digits = 6)

The statistics reveal some information about the distribution of the data in each of the numeric fields, including the number of observations (there are 731 records), the mean, standard deviation, minimum and maximum values, and the quartile values (the threshold values for 25%, 50% - which is also the median, and 75% of the data).

From this, we can see that the mean number of daily rentals is around *848*; but there's a comparatively `large standard deviation`, indicating `a lot of variance` in the number of rentals per day.

We might get a clearer idea of the distribution of rentals values by visualizing the data.
Common plot types for visualizing numeric data distributions are *histograms* and *box plots*, so let's get our `ggplot2` on and create one of each of these for the **rentals** column.

> `patchwork` extends `ggplot` API by providing mathematical operators (such as `+` or `/`) for combining multiple plots.

In [ ]:
library(patchwork)
library(paletteer) # Collection of color palettes
theme_set(theme_light())

# Plot a histogram
hist_plt <- bike_data %>% 
  ggplot(mapping = aes(x = rentals)) + 
  geom_histogram(bins = 100, fill = "midnightblue", alpha = 0.7) +
  
  # Add lines for mean and median
  geom_vline(aes(xintercept = mean(rentals), color = 'Mean'), linetype = "dashed", size = 1.3) +
  geom_vline(aes(xintercept = median(rentals), color = 'Median'), linetype = "dashed", size = 1.3 ) +
  xlab("") +
  ylab("Frequency") +
  scale_color_manual(name = "", values = c(Mean = "red", Median = "yellow")) +
  theme(legend.position = c(0.9, 0.9), legend.background = element_blank())

# Plot a box plot
box_plt <- bike_data %>% 
  ggplot(aes(x = rentals, y = 1)) +
  geom_boxplot(fill = "#E69F00", color = "gray23", alpha = 0.7) +
    # Add titles and labels
  xlab("Rentals")+
  ylab("")


# Combine plots
(hist_plt / box_plt) +
  plot_annotation(title = 'Rental Distribution',
                  theme = theme(plot.title = element_text(hjust = 0.5)))

The plots show that the number of daily rentals ranges from 0 to just over 3,400.
However, the mean (and median) number of daily rentals is closer to the low end of that range, with most of the data between 0 and around 2,200 rentals.
The few values above this are shown in the box plot as small circles, indicating that they are *outliers* - in other words, unusually high or low values beyond the typical range of most of the data.

We can do the same kind of visual exploration of the numeric features.
One way to do this would be to use a `for loop` but ggplot2 provides a way of avoiding this entirely using `facets` 💁.
Facets allow us to create subplots that each display one subset of the data.

This will require us to transform our data into a *long* *format* using `tidyr::pivot_longer`, calculate some statistical summaries and then whip up a histogram for each feature.

In [ ]:
# Create a data frame of numeric features & label
numeric_features <- bike_data %>% 
  select(c(temp, atemp, hum, windspeed, rentals))

# Pivot data to a long format
numeric_features <- numeric_features %>% 
  pivot_longer(!rentals, names_to = "features", values_to = "values") %>%
  group_by(features) %>% 
  mutate(Mean = mean(values),
         Median = median(values))


# Plot a histogram for each feature
numeric_features %>%
  ggplot() +
  geom_histogram(aes(x = values, fill = features), bins = 100, alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free')+
  paletteer::scale_fill_paletteer_d("ggthemes::excel_Parallax") +
  
  # Add lines for mean and median
  geom_vline(aes(xintercept = Mean, color = "Mean"), linetype = "dashed", size = 1.3 ) +
  geom_vline(aes(xintercept = Median, color = "Median"), linetype = "dashed", size = 1.3 ) +
  scale_color_manual(name = "", values = c(Mean = "red", Median = "yellow")) 


The numeric features seem to be more *normally* distributed, with the mean and median nearer the middle of the range of values, coinciding with where the most commonly occurring values are.

> **Note**: The distributions are not *truly* *normal* in the statistical sense, which would result in a smooth, symmetric "bell-curve" histogram with the mean and mode (the most common value) in the center; but they do generally indicate that most of the observations have a value somewhere near the middle.

We've explored the distribution of the `numeric` values in the dataset, but what about the `categorical` features?
These aren't continuous numbers on a scale, so we can't use histograms; but we can plot a bar chart showing the count of each discrete value for each category.

We'll follow the same procedure we used for the numeric feature.

In [ ]:
# Create a data frame of categorical features & label
categorical_features <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit, day, rentals))

# Pivot data to a long format
categorical_features <- categorical_features %>% 
  pivot_longer(!rentals, names_to = "features", values_to = "values") %>%
  group_by(features) %>% 
  mutate(values = factor(values))


# Plot a bar plot for each feature
categorical_features %>%
  ggplot() +
  geom_bar(aes(x = values, fill = features), alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free') +
  paletteer::scale_fill_paletteer_d("ggthemr::solarized") +
  theme(
    panel.grid = element_blank(),
    axis.text.x = element_text(angle = 90))

Many of the categorical features show a more or less *uniform* distribution (meaning there's roughly the same number of rows for each category).
Exceptions to this include:

-   **holiday**: There are many fewer days that are holidays than days that aren't.

-   **workingday**: There are more working days than non-working days.

-   **weathersit**: Most days are category *1* (clear), with category *2* (mist and cloud) the next most common.
    There are comparatively few category *3* (light rain or snow) days, and no category *4* (heavy rain, hail, or fog) days at all.

Now that we know something about the distribution of the data in our columns, we can start to look for relationships between the *features* and the *rentals label* we want to be able to predict.

For the numeric features, we can create scatter plots that show the intersection of feature and label values.

In [ ]:
# Plot a scatter plot for each feature
numeric_features %>% 
  mutate(corr_coef = cor(values, rentals)) %>%
  mutate(features = paste(features, ' vs rentals, r = ', corr_coef, sep = '')) %>% 
  ggplot(aes(x = values, y = rentals, color = features)) +
  geom_point(alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free')+
  paletteer::scale_color_paletteer_d("ggthemes::excel_Parallax")

The *correlation* statistic, *r*, quantifies the apparent relationship.
The correlation statistic is a value between -1 and 1 that indicates the strength of a linear relationship.

In [ ]:
# Calculate correlation coefficient
numeric_features %>% 
  summarise(corr_coef = cor(values, rentals))

The results aren't conclusive, but if you look closely at the scatter plots for `temp` and `atemp`, you can see a `vague diagonal trend` showing that higher rental counts tend to coincide with higher temperatures; and a correlation value of just over 0.5 for both of these features supports this observation.
Conversely, the plots for `hum` and `windspeed` show a `slightly negative correlation`, indicating that there are fewer rentals on days with high humidity or windspeed.

Now let's compare the categorical features to the label.
We'll do this by creating box plots that show the distribution of rental counts for each category.

In [ ]:
# Plot a box plot for each feature
categorical_features %>%
  ggplot() +
  geom_boxplot(aes(x = values, y = rentals, fill = features), alpha = 0.9, show.legend = F) +
  facet_wrap(~ features, scales = 'free') +
  paletteer::scale_fill_paletteer_d("tvthemes::simpsons")+
  theme(
    panel.grid = element_blank(),
    axis.text.x = element_text(angle = 90))

The plots show some variance in the relationship between some category values and rentals.
For example, there's a `clear difference` in the distribution of rentals on weekends (*weekday 0 or 6*) and those during the working week (*weekday 1 to 5*).
Similarly, there are notable differences for `holiday` and `workingday` categories.
There's a noticeable trend that shows different rental distributions in summer and fall months compared to spring and winter months.
The `weathersit` category also seems to make a difference in rental distribution.
The **day** feature we created for the day of the month shows little variation, indicating that it's probably not predictive of the number of rentals.

Amazing!
We have just gone through the phase of **understanding the data**, often referred to as exploratory data analysis (`EDA`).
EDA brings to light how the different variables are related to one another, their distributions, typical ranges, and other attributes.
With these insights in mind, it's time to train some regression models!

### Train a Regression Model using Tidymodels

Now that we've explored the data, it's time to use it to train a regression model that uses the features we've identified as `potentially predictive` to predict the **rentals** label.
The first thing we need to do is create a data frame that contains the predictive features and the label.
Also, we'll need to specify the roles of the predictors.
Are they quantitative (integers/doubles) or are they nominal (characters/factors)?

In [ ]:
# Select desired features and labels
bike_select <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit,
           temp, atemp, hum, windspeed, rentals)) %>% 
  # Encode certain features as categorical
  mutate(across(1:6, factor))

# Get a glimpse of your data
glimpse(bike_select)

Alternatively 🤔, it would have been easier to just deselect the unwanted columns using `select(-c(…))` but we'll leave that for next time.

We *could* train a model using all of the data; but it's common practice in supervised learning to *split* the data into two subsets; a (typically larger) set with which to train the model, and a smaller "hold-back" set with which to validate the trained model.
This enables us to evaluate how well the model performs in order to get a better estimate of how your models will `perform` on `new data`.
It's important to split the data *randomly* (rather than say, taking the first 70% of the data for training and keeping the rest for validation).
This helps ensure that the two subsets of data are `statistically comparable` (so we validate the model with data that has a similar statistical distribution to the data on which it was trained).

To randomly split the data, we'll use `rsample::initial_split()` .
rsample is one of the many packages in the Tidymodels framework.

In [ ]:
# Load the Tidymodels packages
library(tidymodels)

# Split 70% of the data for training and the rest for tesing
set.seed(22)
bike_split <- bike_select %>% 
  initial_split(prop = 0.7,
  # splitting data evenly on the holiday variable
                strata = workingday
  )

# Extract the data in each split
bike_train <- training(bike_split)
bike_test <- testing(bike_split)


cat("Training Set", nrow(bike_train), "rows",
    "\nTest Set", nrow(bike_test), "rows")

this results into the following two datasets:

-   *bike_train*: subset of the dataset used to train the model.

-   *bike_test*: subset of the dataset used to validate the model.

Now ⏲, we're ready to train a model by fitting a suitable regression algorithm to the training data.

Before embarking on more complex machine learning models, it's a good idea to build the simplest possible model to get an idea of what is going on.
We'll use a `linear regression` algorithm, a common starting point for regression that works by trying to find a linear relationship between the $x$ values and the $y$ label.
The resulting model is a function that conceptually defines a line where every possible $x$ and $y$ value combination intersect.

In Tidymodels, you specify models using `parsnip()`.
The goal of [parsnip](https://parsnip.tidymodels.org/) is to provide a tidy, unified interface to models that can be used to try a range of models by specifying three concepts:

-   Model **type** differentiates models such as logistic regression, decision tree models, and so forth.

-   Model **mode** includes common options like regression and classification; some model types support either of these while some only have one mode.

-   Model **engine** is the computational tool which will be used to fit the model.
    Often these are R packages, such as **`"lm"`** or **`"ranger"`**

In tidymodels, we capture that modeling information in a model specification, so setting up your model specification can be a good place to start.

In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")

After a model has been *specified*, the model can be `estimated` or `trained` using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function, typically using a symbolic description of the model (a formula) and some data.

> `rentals ~ .` means we'll fit `rentals` as the predicted quantity, explained by all the predictors/features ie, `.`

In [ ]:
# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Print the model object
lm_mod

So, these are the coefficients that the model *learned* during training.

### Evaluate the Trained Model

It's time to see how the model performed �"�!

How do we do this?
Simple!
Now that we've trained the model, we can use it to predict rental counts for the features we held back in our validation dataset using `parsnip::predict()` like this:

In [ ]:
# Make predictions on test set
pred <- lm_mod %>% 
  predict(new_data = bike_test)

# View predictions
pred %>% 
  slice_head(n = 5)

To make it easier to perform subsequent operations, it would be much convenient to combine the test data and the predictions in a tibble.
Then we can compare these predictions to the actual label values to evaluate how well (or not) the model is working.

In [ ]:
# Predict rentals for the test set and bind it to the test_set
results <- bike_test %>% 
  bind_cols(lm_mod %>% 
    # Predict rentals
    predict(new_data = bike_test) %>% 
      rename(predictions = .pred))

# Compare predictions
results %>% 
  select(c(rentals, predictions)) %>% 
  slice_head(n = 10)

Comparing each prediction with its corresponding "ground truth" actual value isn't a very efficient way to determine how well the model is predicting.
Let's see if we can get a better indication by visualizing a scatter plot that compares the predictions to the actual labels.
We'll also overlay a trend line to get a general sense for how well the predicted labels align with the true labels.

In [ ]:
# Visualise the results
results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(size = 1.6, color = "steelblue") +
  # Overlay a regression line
  geom_smooth(method = "lm", se = F, color = 'magenta') +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

🕵 �"�There's a definite *diagonal trend*, and the intersections of the predicted and actual values are generally following the path of the trend line; but there's a fair amount of difference between the ideal function represented by the line and the results. This variance represents the *residuals* of the model - in other words, the difference between the label predicted when the model applies the coefficients it learned during training to the validation data, and the actual value of the validation label. These residuals when evaluated from the validation data indicate the expected level of *error* when the model is used with new data for which the label is unknown.

You can quantify the residuals by calculating a number of commonly used evaluation metrics.
We'll focus on the following three:

-   `Mean Square Error (MSE)`: The mean of the squared differences between predicted and actual values.
    This yields a relative metric in which the smaller the value, the better the fit of the model

-   `Root Mean Square Error (RMSE)`: The square root of the MSE.
    This yields an absolute metric in the same unit as the label (in this case, numbers of rentals).
    The smaller the value, the better the model (in a simplistic sense, it represents the average number of rentals by which the predictions are wrong)

-   `Coefficient of Determination (usually known as R-squared or R2)`: A relative metric in which the higher the value, the better the fit of the model.
    In essence, this metric represents how much of the variance between predicted and actual label values the model is able to explain.

> `yardstick` is a package in the Tidymodels, used to estimate how well models are working based on the predictions it made for the validation data.
> You can find out more about these and other metrics for evaluating regression models in the [Metric types documentation](https://yardstick.tidymodels.org/articles/metric-types.html).

In [ ]:
# Multiple regression metrics
eval_metrics <- metric_set(rmse, rsq)

# Evaluate RMSE, R2 based on the results
eval_metrics(data = results,
             truth = rentals,
             estimate = predictions)

Good job 🙌!
So now we've quantified the ability of our model to predict the number of rentals.
It definitely has *some* predictive power, but we can probably do better!

### Summary

Here we've explored our data and fit a basic regression model.
In the next notebook, we will try a number of other regression algorithms to improve performance.

### Further Reading

To learn more about Tidymodels, see the [Tidymodels documentation](https://www.tidymodels.org/).

In [ ]:
bike_train %>% 
  ggplot(mapping = aes(x = atemp, y = rentals)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = lm, formula = y ~ x, se = FALSE, color = "lightblue") + 
  
  facet_wrap(vars(season, weekday))

In [ ]:
lr_recipe <- recipe(rentals ~ ., data = bike_train) %>% 
  # Create indicator variables
  step_dummy(all_nominal_predictors()) %>% 
  # Filter out zero variance predictors
  step_nzv(all_predictors()) %>% 
  # Center and scale numeric predictors
  step_normalize(all_numeric_predictors()) %>% 
  step_interact( ~ atemp:starts_with("weekday"):starts_with("season")) %>% 
# Reduce correlation among predictors
  step_corr(all_numeric_predictors()) 

lr_recipe %>% prep()

# Workflow 
lr_wf <- workflow() %>% 
  add_recipe(lr_recipe) %>% 
  add_model(lm_spec)

# Fit workflow
lr_wf_fit = lr_wf %>% 
  fit(data = bike_train)

# Make predictions
pred = lr_wf_fit %>% 
  augment(new_data = bike_test)

eval_metrics(pred, truth = rentals, estimate = .pred)

In [ ]:
lr_recipe <- recipe(rentals ~ ., data = bike_train) %>% 
  # Create indicator variables
  step_dummy(all_nominal_predictors()) %>% 
  # Filter out zero variance predictors
  step_nzv(all_predictors()) %>% 
  # Center and scale numeric predictors
  step_normalize(all_numeric_predictors()) %>% 
  step_interact( ~ atemp:starts_with("weekday"):starts_with("season")) %>% 
# Reduce correlation among predictors
  step_corr(all_numeric_predictors()) 

lr_recipe %>% prep()

# Workflow 
lr_wf <- workflow() %>% 
  add_recipe(lr_recipe) %>% 
  add_model(lm_spec)

# Fit workflow
lr_wf_fit = lr_wf %>% 
  fit(data = bike_train)

# Make predictions
pred = lr_wf_fit %>% 
  augment(new_data = bike_test)

eval_metrics(pred, truth = rentals, estimate = .pred)

## Evaluating model performance using resamples

If the test set should not be used immediately, and repredicting the training set is a bad idea, what should be done?
Resampling methods, such as cross-validation or validation sets, are the solution.

Resampling estimates of performance can generalize to new data in a similar way as estimates from a test set.

Cross-validation involves taking your training set and randomly dividing it up evenly into `V` subsets/folds.
You then use one of the folds for validation and the rest for training, then you repeat these steps with all the subsets and combine the results, usually by taking the mean.
This is just one round of cross-validation.
Sometimes, to obtain better results, data scientists do this more than once, perhaps 5 times.

In [ ]:
set.seed(2056)
# 5 fold CV repeated once
bike_folds <- vfold_cv(data = bike_train, v = 5, repeats = 1)

Now with the resamples, we can evaluate how the `linear_regression workflow` would perform by fitting it across multiple rounds of resampled data

In [ ]:
# Instruct resampling to save the results for each run
ctrl_bike <- control_resamples(save_pred = TRUE)

# Fit models via resampling
bike_res <- fit_resamples(lr_wf, bike_folds, control = ctrl_bike)

# Save resample predictions
lr_bike_preds <- collect_predictions(bike_res)

# Collect resample metrics
bike_res %>% 
  collect_metrics()

We can reliably measure performance using only the **training** data ????

In [ ]:
# Visualise resampled results
lr_bike_preds %>% 
  ggplot(aes(x = rentals, y = .pred)) + 
  geom_point(alpha = .15) +
  geom_abline(color = "dodgerblue") + 
  coord_obs_pred() + 
  ylab("Predicted") +
  theme_light()